In [1]:
folder_path = r"D:\\"
mmap_dir = r"C:\Users\musab\mmap_files_combined_01" 

In [2]:
import os
import numpy as np
import pandas as pd
import random



sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

sample_size = 3

class SensorDataProcessor:
    def __init__(self, folder_path, sensors, mmap_dir, sample_size):
        self.folder_path = folder_path
        self.sensors = sensors
        self.mmap_dir = mmap_dir
        self.sample_size = sample_size

        os.makedirs(mmap_dir, exist_ok=True)

    def _read_single_file(self, file_handle):
        leer = np.frombuffer(file_handle.read(4), dtype=np.single)
        datei = np.frombuffer(file_handle.read(24), dtype=np.uint32)
        file_handle.seek(76)
        lvek = np.frombuffer(file_handle.read(4), dtype=np.uint32)
        file_handle.seek(84 + lvek[0] * 4)
        filetype = np.frombuffer(file_handle.read(4), dtype=np.uint32)

        if filetype != 12:
            t0 = np.frombuffer(file_handle.read(4), dtype=np.single)
            dt = np.frombuffer(file_handle.read(4), dtype=np.single)
            pos_fak = file_handle.tell()
            fak = []

            for n in self.sensors:
                file_handle.seek(pos_fak + 4 * (n - 1))
                fak.append(np.frombuffer(file_handle.read(4), dtype=np.single))

            file_handle.seek(pos_fak + lvek[0] * 4)
            position = file_handle.tell()

            file_handle.seek(0, os.SEEK_END)
            file_size = file_handle.tell()

            record_count = round((file_size - position) / lvek[0] / 2)

            sampled_indices = random.sample(range(record_count), self.sample_size)

            raw_ts_int = np.empty((self.sample_size, len(self.sensors)))

            for i, n in enumerate(sampled_indices):
                for m, sensor_ in enumerate(self.sensors):
                    file_handle.seek(position + 2 * sensor_ + (lvek[0] * 2) * n - 2)
                    raw_ts_int[i, m] = np.frombuffer(file_handle.read(2), dtype=np.int16) * fak[m]

            return raw_ts_int, [f"Sensor {sensor}" for sensor in self.sensors]

        return None

    def _process_files(self, file_paths):
        dfs = []
        sensor_labels = []

        for file_path in file_paths:
            with open(file_path, "rb") as file_handle:
                data = self._read_single_file(file_handle)
                if data:
                    df, labels = data
                    dfs.append(df)
                    sensor_labels.extend(labels)

        combined_array = np.concatenate(dfs, axis=0)

        combined_mmap_path = os.path.join(self.mmap_dir, "combined.mmap")
        combined_mmap = np.memmap(combined_mmap_path, dtype=np.float32, mode="w+", shape=combined_array.shape)
        combined_mmap[:] = combined_array

        return combined_mmap, sensor_labels

    def read_and_process_files(self):
        file_paths = []
        for folder_name in os.listdir(self.folder_path):
            if folder_name.startswith("Yaw"):
                folder_dir = os.path.join(self.folder_path, folder_name)
                for file_name in os.listdir(folder_dir):
                    if file_name.endswith(".int"):
                        file_path = os.path.join(folder_dir, file_name)
                        file_paths.append(file_path)

        result_mmap, sensor_labels = self._process_files(file_paths)
        result_mmap = result_mmap.reshape((-1, len(self.sensors)))

        df = pd.DataFrame(result_mmap, columns=sensor_labels[:len(self.sensors)])
        return df

data_processor = SensorDataProcessor(folder_path, sensors, mmap_dir, sample_size)
df = data_processor.read_and_process_files()

print(df)


         Sensor 1  Sensor 4      Sensor 5      Sensor 6      Sensor 7  \
0        4.108440 -0.343222 -2.486092e-12  3.968278e-12  3.689688e-12   
1        3.672904 -0.468682 -2.257861e-12  3.238590e-12  3.000018e-12   
2        4.002487 -1.789895 -1.535587e-12 -1.241845e-13 -1.906105e-13   
3        3.859261 -2.658597 -2.123352e-12  2.112703e-12  1.935042e-12   
4        3.601032 -1.793389 -1.936909e-12  2.132499e-12  1.956898e-12   
...           ...       ...           ...           ...           ...   
232711  34.680859  0.491862  3.013054e+01  3.013054e+01  3.013054e+01   
232712  23.191164  7.849964  2.659936e+01  2.659936e+01  2.659936e+01   
232713  24.137775  4.227062  2.524690e+01  2.524690e+01  2.524690e+01   
232714  26.205315  5.532192  3.165424e+01  3.165424e+01  3.165424e+01   
232715  22.601553  7.071076  2.469814e+01  2.469814e+01  2.469814e+01   

          Sensor 8     Sensor 9  Sensor 10  Sensor 11    Sensor 12  ...  \
0       284.564789   618.042969   7.775018   1.4